In [1]:
import csv
import numpy as np
import pandas as pd
import json
from functools import reduce

In [2]:
# data = pd.read_csv("TestData/MockData2-goodNames-normalized.csv") 
# data = pd.read_csv("TestData/COVID.csv")
# data1 = pd.read_csv("TestData/1_ToyDataset-All.csv") 
# data2 = pd.read_csv("TestData/2_ToyDataset-NoEdges.csv") 
data1 = pd.read_csv("TestData/1_ToyDataset-All copy.csv") 
data2 = pd.read_csv("TestData/1_ToyDataset-All copy 2.csv") 

In [3]:
num_data = 2

In [4]:
# GOAL
{
    "ids":
        ["PAK2", "HRI", "P20152", "Q01986", "P01237"],

    "nodes": 
        [{"id": "PAK2", "group": "1", "states":[{"source_attr_1": -1, "source_attr_2": 3}, 
                                                {"source_attr_1": -5, "source_attr_2": 5}]},
        {"id": "HRI", "group": "1", "states":[{"source_attr_1": 3, "source_attr_2": 1},
                                              {"source_attr_1": -4, "source_attr_2": 1}]},
        {"id": "P20152", "group": "2", "states": [{"target_attr_2": 0.01, "target_attr_1": [{"ph7" : 3.5}, {"ph206" : -5.2}, {"ph306": -7.5}]},
                                                  {"target_attr_2": 2, "target_attr_1": [{"ph7" : -3}, {"ph206" : 1}, {"ph306": 0.2}]}]},
        {"id": "Q01986", "group": "2", "states":[{"target_attr_2": -1, "target_attr_1": [{"ph7" : 2.1}, {"ph206" : -3.9}]},
                                                {"target_attr_2": 4, "target_attr_1": [{"ph7" : -4}]}]},
        {"id": "P01237", "group": "2", "states":[{"target_attr_2": 2, "target_attr_1": [{}]}]}],

    "links": 
        [{"source": "PAK2", "target": "P20152", "edge_attr": [{"dataset":0, "edge_attr_1": -1, "edge_attr_2": 3}, 
                                                              {"dataset":1, "edge_attr_1": -5, "edge_attr_2": 5}]}, 
        {"source": "PAK2", "target": "Q01986", "edge_attr":[{"dataset":0, "edge_attr_1": -1, "edge_attr_2": 3}, 
                                                              {"dataset":1, "edge_attr_1": -5, "edge_attr_2": 5}]},  
        {"source": "HRI", "target": "P01237", "edge_attr": [{"dataset":0, "edge_attr_1": -1, "edge_attr_2": 3}, 
                                                              {"dataset":1, "edge_attr_1": -5, "edge_attr_2": 5}]}]
        }    
 

IndentationError: unexpected indent (<ipython-input-4-b556c3d3c4e1>, line 3)

In [24]:
datasets = [data1, data2]

In [5]:
def normalize(values, actual_bounds, desired_bounds):
    final = []
    for i in values:
        final.append(desired_bounds[0] + (i - actual_bounds[0]) * (desired_bounds[1] - desired_bounds[0]) / (actual_bounds[1] - actual_bounds[0]))
    return final

In [6]:
def attribute_dict (dataframe, col_name_source_or_target, col_name_attribute_of_interest):
    dictionary = {}
    
    min_val = min(dataframe[col_name_attribute_of_interest].values)
    max_val = max(dataframe[col_name_attribute_of_interest].values)
    actual_bounds = (min_val, max_val)
    desired_bounds = (0, 1)
    
    for i in range(len(dataframe)):
        if col_name_attribute_of_interest in dataframe.columns:
            value = dataframe[col_name_attribute_of_interest][i]
            dictionary[dataframe[col_name_source_or_target][i]] = normalize([value], actual_bounds, desired_bounds)[0]
        else:
            dictionary[dataframe[col_name_source_or_target][i]] = None
            
    return dictionary

In [7]:
def partitions_dict (dataframe, col_name_target, col_name_partitions, col_name_attribute_of_interest):
    partitions = {}
    rep = {}

    target = dataframe[col_name_target]
    val = dataframe[col_name_attribute_of_interest]

    if col_name_partitions in dataframe.columns:
        site = dataframe[col_name_partitions]
        for i in range(len(dataframe)):
            if target[i] not in partitions:
                partitions[target[i]] = [{site[i]: val[i]}]
                rep[target[i]] = [site[i]]

            else:
                if site[i] not in rep[target[i]]:
                    partitions[target[i]] += [{site[i]: val[i]}]
                    rep[target[i]] += [site[i]]
    
    return partitions    

In [8]:
def links_info (dataframe, col_name_source, col_name_target):
    links = []

    for i in range(len(dataframe)):
        if [dataframe[col_name_source][i], dataframe[col_name_target][i]] not in links:
            links.append([dataframe[col_name_source][i], dataframe[col_name_target][i]])
    return links

In [23]:
links

[['Source1', 'Target1'],
 ['Source1', 'Target2'],
 ['Source1', 'Target3'],
 ['Source1', 'Target4'],
 ['Source1', 'Target5'],
 ['Source1', 'Target6'],
 ['Source2', 'Target1'],
 ['Source2', 'Target7'],
 ['Source3', 'Target8'],
 ['Source3', 'Target3'],
 ['Source3', 'Target9'],
 ['Source3', 'Target10'],
 ['Source3', 'Target11'],
 ['Source3', 'Target12'],
 ['Source3', 'Target13'],
 ['Source3', 'Target14'],
 ['Source3', 'Target15'],
 ['Source3', 'Target16'],
 ['Source4', 'Target17'],
 ['Source4', 'Target18'],
 ['Source4', 'Target19'],
 ['Source4', 'Target20'],
 ['Source5', 'Target21'],
 ['Source5', 'Target22'],
 ['Source5', 'Target23'],
 ['Source5', 'Target17'],
 ['Source5', 'Target5'],
 ['Source6', 'Target24'],
 ['Source6', 'Target25'],
 ['Source6', 'Target26'],
 ['Source6', 'Target23']]

In [9]:
def sorce_states (datasets): #can be made for source and targets
    final = {}
    ks = []
    
    for j in datasets:
        ks += list(j["SourceID"])
    ks = set(ks)
    for i in range(len(datasets)):
        source_att_1 = attribute_dict(datasets[i], "SourceID", "SourceNodeColor")
        source_att_2 = attribute_dict(datasets[i], "SourceID", "SourceNodeSize")
        for k in ks:
            if k in list(datasets[i]["SourceID"]):
                if k in final:
                    final[k] += [{"dataset":i, "source_attr_1":source_att_1[k], "source_attr_2": source_att_2[k]}]
                else:
                    final[k] = [{"dataset":i, "source_attr_1":source_att_1[k], "source_attr_2": source_att_2[k]}]
            else:
                if k not in final:
                    final[k] = []

    return final

In [10]:
def target_states_NoPartitions (datasets):
    final = {}
    ks = []
    
    for j in datasets:
        ks += list(j["TargetID"])
    ks = set(ks)

    for i in range(len(datasets)):
        target_att_1 = attribute_dict(datasets[i], "TargetID", "TargetNodeColor")
        target_att_2 = attribute_dict(datasets[i], "TargetID", "TargetNodeSize")
        
        for k in ks:
            if k in list(datasets[i]["TargetID"]):
                if k in final:
                    final[k] += [{"dataset":i, "target_att_1":target_att_1[k], "target_att_2": target_att_2[k]}]
                else:
                    final[k] = [{"dataset":i, "target_att_1":target_att_1[k], "target_att_2": target_att_2[k]}]
            else:
                if k not in final:
                    final[k] = []

    return final

In [11]:
def target_states_Partitions (datasets):
    final = {}
    ks = []
    
    for j in datasets:
        ks += list(j["TargetID"])
    ks = set(ks)

    for i in range(len(datasets)):
        target_att_2 = attribute_dict(datasets[i], "TargetID", "TargetNodeSize")
        partitions = partitions_dict(datasets[i], "TargetID", "PhosphorylationSiteID", "TargetNodeColor")
        
        for k in ks:
            if k in list(datasets[i]["TargetID"]):
                if k in final:
                    final[k] += [{"dataset":i, "target_att_1":partitions[k], "target_att_2": target_att_2[k]}]
                else:
                    final[k] = [{"dataset":i, "target_att_1":partitions[k], "target_att_2": target_att_2[k]}]
            else:
                if k not in final:
                    final[k] = []

    return final

In [12]:
source_att = sorce_states(datasets) #STATE #ID
target_states_NoPartitions_d = target_states_NoPartitions(datasets) #STATE #ID
target_states_Partitions_d = target_states_Partitions(datasets) #STATE #ID

In [14]:
source_att

{'Source4': [{'dataset': 0,
   'source_attr_1': 0.0,
   'source_attr_2': 0.2555066079295154},
  {'dataset': 1, 'source_attr_1': 0.0, 'source_attr_2': 0.2555066079295154}],
 'Source6': [{'dataset': 0,
   'source_attr_1': 0.8405253283302064,
   'source_attr_2': 0.3920704845814977},
  {'dataset': 1,
   'source_attr_1': 0.8405253283302064,
   'source_attr_2': 0.3920704845814977}],
 'Source3': [{'dataset': 0,
   'source_attr_1': 0.7392120075046904,
   'source_attr_2': 0.13656387665198239}],
 'Source5': [{'dataset': 0,
   'source_attr_1': 0.7450281425891182,
   'source_attr_2': 1.0},
  {'dataset': 1, 'source_attr_1': 0.7450281425891182, 'source_attr_2': 1.0}],
 'Source1': [{'dataset': 0,
   'source_attr_1': 0.14634146341463417,
   'source_attr_2': 0.0},
  {'dataset': 1, 'source_attr_1': 0.14634146341463417, 'source_attr_2': 0.0}],
 'Source2': [{'dataset': 0,
   'source_attr_1': 1.0,
   'source_attr_2': 0.5044052863436124},
  {'dataset': 1, 'source_attr_1': 1.0, 'source_attr_2': 0.50440528634

In [15]:
target_states_Partitions_d["Target17"]

[{'dataset': 0,
  'target_att_1': [{'SiteTarget17-1': -0.46},
   {'SiteTarget17-2': 1.35},
   {'SiteTarget17-3': -3.58},
   {'SiteTarget17-4': -0.67},
   {'SiteTarget17-5': -0.38},
   {'SiteTarget17-6': 3.38}],
  'target_att_2': 0.9510309278350516},
 {'dataset': 1,
  'target_att_1': [{'SiteTarget17-1': -0.46},
   {'SiteTarget17-2': 1.35},
   {'SiteTarget17-3': -3.58},
   {'SiteTarget17-4': -0.67},
   {'SiteTarget17-5': -0.38},
   {'SiteTarget17-6': 3.38}],
  'target_att_2': 0.9510309278350516}]

In [16]:
target_states_Partitions

<function __main__.target_states_Partitions(datasets)>

In [17]:
# source_att_1 = attribute_dict(data1, "SourceID", "SourceNodeColor")
# source_att_2 = attribute_dict(data1, "SourceID", "SourceNodeSize")
# target_att_1 = attribute_dict(data1, "TargetID", "TargetNodeColor")
# target_att_2 = attribute_dict(data1, "TargetID", "TargetNodeSize")
edge_att_1 = attribute_dict(data1, "TargetID", "EdgeColor")
edge_att_2 = attribute_dict(data1, "TargetID", "EdgeWeight")

# partitions = partitions_dict(data1, "TargetID", "PhosphorylationSiteID", "TargetNodeColor")
links = links_info(data1, "SourceID", "TargetID")

In [27]:
def edge_states (datasets): #can be made for source and targets
    final = {}
    ks = []
    
    for j in datasets:
        ks += list(j["TargetID"])
    ks = set(ks)
    for i in range(len(datasets)):
        edge_att_1 = attribute_dict(datasets[i], "TargetID", "EdgeColor")
        edge_att_2 = attribute_dict(datasets[i], "TargetID", "EdgeWeight")
        for k in ks:
            if k in list(datasets[i]["TargetID"]):
                if k in final:
                    final[k] += [{"dataset":i, "edge_att_1":edge_att_1[k], "edge_att_2": edge_att_2[k]}]
                else:
                    final[k] = [{"dataset":i, "edge_att_1":edge_att_1[k], "edge_att_2": edge_att_2[k]}]
            else:
                if k not in final:
                    final[k] = []

    return final

In [28]:
edge_states (datasets)

{'Target6': [{'dataset': 0,
   'edge_att_1': 0.11871508379888263,
   'edge_att_2': 0.36814621409921666},
  {'dataset': 1,
   'edge_att_1': 0.08815028901734102,
   'edge_att_2': 0.36814621409921666}],
 'Target12': [{'dataset': 0,
   'edge_att_1': 0.9441340782122905,
   'edge_att_2': 0.5378590078328982}],
 'Target2': [{'dataset': 0,
   'edge_att_1': 0.5111731843575419,
   'edge_att_2': 0.15404699738903394},
  {'dataset': 1,
   'edge_att_1': 0.49421965317919075,
   'edge_att_2': 0.15404699738903394}],
 'Target17': [{'dataset': 0,
   'edge_att_1': 0.8659217877094971,
   'edge_att_2': 1.0},
  {'dataset': 1, 'edge_att_1': 0.861271676300578, 'edge_att_2': 1.0}],
 'Target7': [{'dataset': 0,
   'edge_att_1': 0.03351955307262566,
   'edge_att_2': 0.3472584856396867},
  {'dataset': 1, 'edge_att_1': 0.0, 'edge_att_2': 0.3472584856396867}],
 'Target8': [{'dataset': 0,
   'edge_att_1': 0.803072625698324,
   'edge_att_2': 0.216710182767624}],
 'Target19': [{'dataset': 0,
   'edge_att_1': 0.4371508379

In [18]:
# testing_a_null_column = partitions_dict(data, "TargetID", "hello", "TargetNodeColor")
# testing_a_null_column

In [19]:
# dictionary = {"nodes": [], "links": []}

# source = data["SourceID"].unique()
# target = data["TargetID"].unique()

# for s in source:
#     dictionary['nodes'] += [{'id': s, "group": 1, "source_attr_1": source_att_1[s], "source_attr_2": source_att_1[s]}]
    
# for t in target:
#     dictionary['nodes'] += [{'id': t, "group": 2, "target_attr_2": target_att_2[t], "positions": partitions[t]}]
    
# for s, t in links:
#     dictionary['links'] += [{"source": s, "target": t, "edge_attr_1": edge_att_1[t], "edge_attr_2": edge_att_2[t]}]

In [20]:
dictionary = {"nodes": [], "links": []}

# source = data["SourceID"].unique()
# target = data["TargetID"].unique()

source = []    
for j in datasets:
    source += list(j["SourceID"])
source = set(source)

target = []    
for j in datasets:
    target += list(j["TargetID"])
target = set(target)


for s in source:
    dictionary['nodes'] += [{'id': s, "group": 1, "states": source_att[s]}]
    
for t in target:
    dictionary['nodes'] += [{'id': t, "group": 2, "state": target_states_Partitions_d[t]}]
    
for s, t in links:
    dictionary['links'] += [{"source": s, "target": t, "edge_attr_1": edge_att_1[t], "edge_attr_2": edge_att_2[t]}]

In [248]:
with open('mockdata.json', 'w') as fp:
    json.dump(dictionary, fp)

In [21]:
dictionary

{'nodes': [{'id': 'Source4',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.0,
     'source_attr_2': 0.2555066079295154},
    {'dataset': 1,
     'source_attr_1': 0.0,
     'source_attr_2': 0.2555066079295154}]},
  {'id': 'Source6',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.8405253283302064,
     'source_attr_2': 0.3920704845814977},
    {'dataset': 1,
     'source_attr_1': 0.8405253283302064,
     'source_attr_2': 0.3920704845814977}]},
  {'id': 'Source3',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.7392120075046904,
     'source_attr_2': 0.13656387665198239}]},
  {'id': 'Source5',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.7450281425891182,
     'source_attr_2': 1.0},
    {'dataset': 1,
     'source_attr_1': 0.7450281425891182,
     'source_attr_2': 1.0}]},
  {'id': 'Source1',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.14634146341463417,
     'source_attr_2': 0.0},

In [45]:
dictionary

{'nodes': [{'id': 'Source2',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 1.0,
     'source_attr_2': 0.5044052863436124},
    {'dataset': 1,
     'source_attr_1': 1.0,
     'source_attr_2': 0.5044052863436124}]},
  {'id': 'Source1',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.14634146341463417,
     'source_attr_2': 0.0},
    {'dataset': 1,
     'source_attr_1': 0.14634146341463417,
     'source_attr_2': 0.0}]},
  {'id': 'Source6',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.8405253283302064,
     'source_attr_2': 0.3920704845814977},
    {'dataset': 1,
     'source_attr_1': 0.8405253283302064,
     'source_attr_2': 0.3920704845814977}]},
  {'id': 'Source3',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.7392120075046904,
     'source_attr_2': 0.13656387665198239}]},
  {'id': 'Source5',
   'group': 1,
   'states': [{'dataset': 0,
     'source_attr_1': 0.7450281425891182,
     'source_attr_2': 1.0}